In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 747.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset
import json
with open("/content/drive/MyDrive/dl_dataset/ian_dataset_cleaned.json", "r", encoding="utf-8") as f:
    data1 = json.load(f)

with open("/content/drive/MyDrive/dl_dataset/stan_dataset_cleaned.json", "r", encoding="utf-8") as f:
    data2 = json.load(f)

with open("/content/drive/MyDrive/dl_dataset/jona_dataset_cleaned.json", "r", encoding="utf-8") as f:
    data3 = json.load(f)
json_data = data1 + data2 + data3

In [ ]:
empty_data = {
    "pokemon": "",
    "input": "Write a detailed 'short_summary' section for the pokemon: "" using only the data given.\n\n<DATA></DATA>",
    "output": "<OUTPUT>There is insufficient data to generate a summary.</OUTPUT>"
}

json_data.extend([empty_data.copy() for _ in range(75)])


In [ ]:
SYSTEM_PROMPT = (
    "You are a competitive Pokémon VGC analysis model.\n"
    "You must generate analysis strictly from the information provided in the <DATA> block.\n"
    "Do not use outside knowledge or assumptions.\n"
    "If the data is missing, ambiguous, or does not match the requested Pokémon, "
    "respond with a refusal.\n"
    "Follow the requested format exactly."
)
text_data = []
for data in json_data:
    pokemon = data["pokemon"].strip()
    input_body = data["input"].strip()
    output_text = data["output"].strip()

    # Canonical, future-proof instruction format
    user_prompt = (
        f"Write a detailed short_summary for the Pokémon: {pokemon}.\n"
        f"Use only the data provided below.\n\n"
        f"{input_body}"
    )

    text = (
        f"<|system|>{SYSTEM_PROMPT}<|end|>"
        f"<|user|>{user_prompt}<|end|>"
        f"<|assistant|>{output_text}<|end|>"
    )

    text_data.append({"text": text})

train_dataset = Dataset.from_list(text_data).shuffle(seed=3407)
del text_data

In [ ]:
train_dataset.save_to_disk("/content/drive/MyDrive/dl_dataset/train_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/2209 [00:00<?, ? examples/s]

In [ ]:
# view dataset
train_dataset = Dataset.load_from_disk("/content/drive/MyDrive/dl_dataset/train_dataset")

In [ ]:
from unsloth import FastLanguageModel
import os
os.environ['UNSLOTH_USE_MODELSCOPE'] = '1'

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048
dtype=None

model, tokenizer = FastLanguageModel.from_pretrained(model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

# look at model internals
print(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.5: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32009)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((3072,), eps=1e-05)
        (post_attention_laye

In [ ]:
# Define fine-tuning parameters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha =  16,
    lora_dropout=0.0,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=123,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.12.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer, SFTConfig
from unsloth import is_bf16_supported
from transformers import TrainingArguments
import matplotlib.pyplot as plt

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2, # number of cpu processes for preprocessing
    packing=False, # boolean to pack multiple short inputs into one long input
    args=TrainingArguments(
        per_device_train_batch_size=2, # how many samples per gpu pass, higher -> higher VRAM
        gradient_accumulation_steps=8, # collects gradients from multiple steps before gradient update
        warmup_steps=50, # dampener for learning rate (slow down lr to avoid instabiltiy during training ( exploding gradient ))
        max_steps=500, # more steps = better training (increased from 350)
        learning_rate=3e-4,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_steps=100,
        save_total_limit=3,
        report_to="none",
        gradient_checkpointing=True
    ),
)

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,209 | Num Epochs = 4 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)


Step,Training Loss
10,2.702400
20,2.472100
30,2.028900
40,1.660200
50,1.523600
60,1.424900
70,1.365000
80,1.305000
90,1.277700
100,1.229800


Step,Training Loss
10,2.702400
20,2.472100
30,2.028900
40,1.660200
50,1.523600
60,1.424900
70,1.365000
80,1.305000
90,1.277700
100,1.229800


In [ ]:
model.save_pretrained("/content/drive/MyDrive/phi3-vgc-hf")
tokenizer.save_pretrained("/content/drive/MyDrive/phi3-vgc-hf")

('/content/drive/MyDrive/phi3-vgc-hf/tokenizer_config.json',
 '/content/drive/MyDrive/phi3-vgc-hf/special_tokens_map.json',
 '/content/drive/MyDrive/phi3-vgc-hf/chat_template.jinja',
 '/content/drive/MyDrive/phi3-vgc-hf/tokenizer.model',
 '/content/drive/MyDrive/phi3-vgc-hf/added_tokens.json',
 '/content/drive/MyDrive/phi3-vgc-hf/tokenizer.json')

In [ ]:
train_dataset.data

MemoryMappedTable
text: string
----
text: [["<|system|>You are a competitive Pokémon VGC analysis model.
You must generate analysis strictly f (... 1912 chars omitted)","<|system|>You are a competitive Pokémon VGC analysis model.
You must generate analysis strictly f (... 1721 chars omitted)","<|system|>You are a competitive Pokémon VGC analysis model.
You must generate analysis strictly f (... 1817 chars omitted)","<|system|>You are a competitive Pokémon VGC analysis model.
You must generate analysis strictly f (... 1903 chars omitted)","<|system|>You are a competitive Pokémon VGC analysis model.
You must generate analysis strictly f (... 1843 chars omitted)",...,"<|system|>You are a competitive Pokémon VGC analysis model.
You must generate analysis strictly f (... 2053 chars omitted)","<|system|>You are a competitive Pokémon VGC analysis model.
You must generate analysis strictly f (... 2176 chars omitted)","<|system|>You are a competitive Pokémon VGC analysis model.
You must generat

In [ ]:
model_path = "/content/drive/MyDrive/vgcmaster_lora_model"
tokenizer_path = "/content/drive/MyDrive/vgcmaster_model"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

('/content/drive/MyDrive/vgcmaster_model/tokenizer_config.json',
 '/content/drive/MyDrive/vgcmaster_model/special_tokens_map.json',
 '/content/drive/MyDrive/vgcmaster_model/chat_template.jinja',
 '/content/drive/MyDrive/vgcmaster_model/tokenizer.model',
 '/content/drive/MyDrive/vgcmaster_model/added_tokens.json',
 '/content/drive/MyDrive/vgcmaster_model/tokenizer.json')

In [ ]:
trainer_stats.training_loss

1.0893040962219238

In [ ]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer
from unsloth import FastLanguageModel

# Ensure inference mode
FastLanguageModel.for_inference(model)

# IMPORTANT: use the Phi-3 chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3",
)

# Validation prompt: EMPTY DATA CASE
messages = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT,
    },
    {
        "role": "user",
        "content": (
            "Write a detailed short_summary for the Pokémon: Dragonite.\n"
            "Use only the data provided below.\n\n"
            "<DATA></DATA>"
        ),
    },
]

# Apply chat template correctly
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Stream output
streamer = TextStreamer(tokenizer, skip_prompt=True)

# Generate
_ = model.generate(
    inputs,
    streamer=streamer,
    max_new_tokens=128,
    temperature=0.2,
)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<OUTPUT>Dragonite is a premier physical attacker in VGC, leveraging its blistering 105 base Speed to outpace the crowded 100-110 tier and immediately threaten opposing Dragons, Ground-types, and bulky Steels. Its 100 base Attack and access to STAB Dragon Darts or Ice Beam give it a respectable 120-130 BP, letting it 2HKO most neutral targets after minor chip. The 100/110/100/1


In [ ]:
correct_data = """
<DATA>
{
  "Dragonite": {
    "Abilities": {"multiscale": 0.92},
    "Moves": ["dragondance", "extremespeed", "protect"],
    "Items": {"lumberry": 0.45},
    "Teammates": ["Chien-Pao", "Rillaboom"]
  }
}
</DATA>
"""

messages = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT,
    },
    {
        "role": "user",
        "content": (
            "Write a detailed short_summary for the Pokémon: Dragonite.\n"
            "Use only the data provided below.\n\n"
            f"{correct_data}"
        ),
    },
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

streamer = TextStreamer(tokenizer, skip_prompt=True)

_ = model.generate(
    inputs,
    streamer=streamer,
    max_new_tokens=256,
    temperature=0.3,
)

<OUTPUT>Dragonite is a premier support Pokémon in VGC, leveraging Multiscale to survive hits that would otherwise KO it, while Dragon Dance turns it into a 120 BP Dragon-type that can outspeed and KO threats with Extreme Speed. Its EV spread is optimized for bulk and speed, and it is almost always seen with Lumberberry to heal off chip and maintain its physical bulk. The combination of Dragon Dance, Extreme Speed, and Protect makes it a premier pivot that can both support and pressure opposing teams.</OUTPUT><|end|>
